In [4]:
import pandas as pd
from util.general import load_csv_pandas

In [30]:
df = load_csv_pandas('원천조사가격정보_201601.csv')
df = df[['조사일자', '조사구분명', '표준품목명', '조사가격품목명', '표준품종명', '조사지역명', '표준시장명']]
print(df[(df.조사구분명 == '소비자가격') & (df.표준품목명 == '고등어류')])
print(df)

             조사일자  조사구분명 표준품목명 조사가격품목명  표준품종명 조사지역명     표준시장명
3259   2016-01-04  소비자가격  고등어류     고등어  망치고등어  서울서부      경동시장
3260   2016-01-04  소비자가격  고등어류     고등어  망치고등어  서울서부     복조리시장
3261   2016-01-04  소비자가격  고등어류     고등어  망치고등어  서울서부  영등포 유통상가
3262   2016-01-04  소비자가격  고등어류     고등어  망치고등어    서울   이마트 수색점
3263   2016-01-04  소비자가격  고등어류     고등어  망치고등어    부산      부전시장
...           ...    ...   ...     ...    ...   ...       ...
74970  2016-01-29  소비자가격  고등어류     고등어  망치고등어    전주       이마트
74971  2016-01-29  소비자가격  고등어류     고등어  망치고등어  전남순천  홈플러스 순천점
74972  2016-01-29  소비자가격  고등어류     고등어  망치고등어  경북안동   이마트 안동점
74973  2016-01-29  소비자가격  고등어류     고등어  망치고등어  경남창원   이마트 창원점
74974  2016-01-29  소비자가격  고등어류     고등어  망치고등어    제주  이마트 신제주점

[1300 rows x 7 columns]
             조사일자        조사구분명 표준품목명 조사가격품목명         표준품종명  조사지역명    표준시장명
0      2016-01-01  수산물산지가격(수협)  고등어류     고등어         망치고등어  경남삼천포   삼천포(위)
1      2016-01-01  수산물산지가격(수협)   갈치류      갈치           은갈치  경남삼천포   삼천포(위)
2     

In [12]:
metropolitan = ['서울서부', '대전', '부산', '대구', '광주', '인천', '울산', '제주']